In [1]:
from autobahn.asyncio import component as ab_utils
from autobahn.wamp import types as ab_types
from collections import deque, defaultdict


class SignalReceiver(object):
    def __init__(self, host='plfelt-mbp.local', wamp_port=5555, maxsize=1000):
        self.host = host
        self.wamp_port = wamp_port
        self.q = defaultdict(deque)
        super().__init__()
        
    def _get_wamp_component(self, host, wamp_port):
        return ab_utils.Component(
            transports=u"ws://{host}:{port}/ws".format(host=host, port=wamp_port),
            realm=u"realm1",
        )

    def handle_signal(self, *args, **kwargs):
#        print('got sensor data',len(args),kwargs.keys())
        if kwargs is not None and 'robot_name' in kwargs:
            self.q[kwargs['robot_name'], kwargs['signal_type'], kwargs['name']].append((args, kwargs))
#             self.q[kwargs['robot_name']].append((args, kwargs))
    
    def on_join(self, session, details):
        session.subscribe(self.handle_signal, '', options=ab_types.SubscribeOptions(match='prefix'))
    
    def start(self):
        self.wamp_component = self._get_wamp_component(self.host, self.wamp_port)
        self.wamp_component.on_join(self.on_join)
        self.wamp_component.start()

    def stop(self):
        self.wamp_component.stop()

In [2]:
# Listen for events
signal_receiver = SignalReceiver()
q = signal_receiver.q
signal_receiver.start()

Using default loop


In [8]:
len(q)

2

In [19]:
q.keys()

dict_keys([('marion', 'sensor', 'picam')])

Exception in callback _SelectorTransport._call_connection_lost(None)
handle: <Handle _SelectorTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "/usr/lib/python3.5/asyncio/events.py", line 125, in _run
    self._callback(*self._args)
  File "/usr/lib/python3.5/asyncio/selector_events.py", line 606, in _call_connection_lost
    self._protocol.connection_lost(exc)
  File "/usr/local/lib/python3.5/dist-packages/autobahn/asyncio/component.py", line 308, in lost
    rtn = orig(fail)
  File "/usr/local/lib/python3.5/dist-packages/autobahn/asyncio/websocket.py", line 100, in connection_lost
    self._connectionLost(exc)
  File "/usr/local/lib/python3.5/dist-packages/autobahn/websocket/protocol.py", line 3363, in _connectionLost
    WebSocketProtocol._connectionLost(self, reason)
  File "/usr/local/lib/python3.5/dist-packages/autobahn/websocket/protocol.py", line 1096, in _connectionLost
    reason_string = _maybe_tls_reason(reason.value)
AttributeError: 'NoneTyp

In [10]:
mq = q['marion', 'sensor', 'picam']

In [23]:
len(mq)

15635